In [22]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNet
import zipfile

In [6]:
zip_path = "/AGRINOVA-CAPSTONE/dataset/archive.zip"
extract_path = "/AGRINOVA-CAPSTONE/dataset/tomato/"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
    zip_ref.close()

In [33]:
train_dir = "/AGRINOVA-CAPSTONE/dataset/tomato/train/"
val_dir = "/AGRINOVA-CAPSTONE/dataset/tomato/valid/"

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=128,
    class_mode='categorical'
)

val_generator = validation_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=128,
    class_mode='categorical'
)

Found 25851 images belonging to 11 classes.
Found 6684 images belonging to 11 classes.


In [34]:
#transver learning 
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

In [44]:
base_model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

In [41]:
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=128, activation='relu'))
model.add(tf.keras.layers.Dense(units=64, activation='relu'))
model.add(tf.keras.layers.Dense(11, activation='softmax'))

In [42]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(1e-4),
              metrics=['accuracy'])

In [43]:
history = model.fit(
    train_generator,
    steps_per_epoch=8,
    epochs=20,
    validation_data=val_generator,
    validation_steps=8
)

Epoch 1/20
8/8 [==============================] - 64s 8s/step - loss: 3.9933 - accuracy: 0.1543 - val_loss: 2.4399 - val_accuracy: 0.1670
Epoch 2/20
8/8 [==============================] - 77s 10s/step - loss: 2.1032 - accuracy: 0.2939 - val_loss: 1.8322 - val_accuracy: 0.3906
Epoch 3/20
8/8 [==============================] - 67s 9s/step - loss: 1.9419 - accuracy: 0.3398 - val_loss: 1.8282 - val_accuracy: 0.4199
Epoch 4/20
8/8 [==============================] - 66s 9s/step - loss: 1.6478 - accuracy: 0.4652 - val_loss: 1.5656 - val_accuracy: 0.5029
Epoch 5/20
8/8 [==============================] - 62s 8s/step - loss: 1.7148 - accuracy: 0.4248 - val_loss: 1.2626 - val_accuracy: 0.5898
Epoch 6/20
8/8 [==============================] - 61s 8s/step - loss: 1.4590 - accuracy: 0.5195 - val_loss: 1.7100 - val_accuracy: 0.4834
Epoch 7/20


KeyboardInterrupt: 